In [5]:
import datetime
import load_data
import pandas as pd
import numpy as np

In [103]:
#### Only used in PrepareData.ipynb ####
from load_data import *

def get_single_df(df_name, i):
    csvs = ['csv/'+ f for f in list_s3_dir('csv/{0}__{1}'.format(df_name, i))]
    dfs = []
    for f in csvs:
        df = s3_to_df(f)
        print(len(dfs), end = ' ')
        dfs.append(df)
    return pd.concat(dfs)

def combine_notebooks(workers = list(range(10))):
    start = datetime.datetime.now()
    for i in workers:
        start_i = datetime.datetime.now()
        nb_i = get_single_df('notebooks2',i)
        nb1_i = s3_to_df('csv/notebooks1_{0}.csv'.format(i))
        nb_i = nb_i.merge(nb1_i, on = 'file')
        mid_i = datetime.datetime.now()
        print('\nnotebooks_{0} loaded in'.format(i),mid_i - start_i)
        nb_i.to_csv('data_final/notebooks_{0}.csv'.format(i), index = False)
        end_i = datetime.datetime.now() 
        print('notebooks_{0} in csv in'.format(i), end_i-mid_i)
    
    end = datetime.datetime.now()
    print('DONE! Now all in csv.', end - start)
    
def combine_cells(workers = list(range(10))):
    start = datetime.datetime.now()
    for i in workers:
        start_i = datetime.datetime.now()
        cells_i = get_single_df('cells1',i)
        mid_i = datetime.datetime.now()
        print('\ncells_{0} loaded in'.format(i),mid_i - start_i)
        
        cells_i.to_csv('data_final/cells_{0}.csv'.format(i), index = False)
        end_i = datetime.datetime.now()
        print('cells_{0} in csv in'.format(i), end_i - mid_i)
     
    end = datetime.datetime.now()
    print('DONE! Now all in csv.', end - start)
    
def combine_repos(workers = list(range(10))):
    start = datetime.datetime.now()
    for i in workers:
        start_i = datetime.datetime.now()
        repos_i = s3_to_df('csv/repos2_{0}.csv'.format(i))
        mid_i = datetime.datetime.now()
        print('repos_{0} loaded in'.format(i),mid_i - start_i)
        repos_i.to_csv('data_final/repos_{0}.csv'.format(i), index = False)
        end_i = datetime.datetime.now()
        print('repos_{0} in csv in'.format(i), end_i - mid_i)
        
    end = datetime.datetime.now()
    print('DONE! Now all in csv.', end - start)
    
def combine_owners(workers = list(range(10))):
    start = datetime.datetime.now()
    for i in workers:
        start_i = datetime.datetime.now()
        repos_i = s3_to_df('csv/owners2_{0}.csv'.format(i))
        mid_i = datetime.datetime.now()
        print('owners_{0} loaded in'.format(i),mid_i - start_i)
        repos_i.to_csv('data_final/owners_{0}.csv'.format(i), index = False)
        end_i = datetime.datetime.now()
        print('owners_{0} in csv in'.format(i), end_i - mid_i)
        
    end = datetime.datetime.now()
    print('DONE! Now all in csv.', end - start)

# Data was extracted in 10 different sets of data. First step is to combine them all

## Owners

Download all owners csv files from s3

In [ ]:
combine_owners()


Concatenate these files

In [54]:
!head -1 data_final/owners_0.csv > data_final/owners.csv

In [55]:
!for filename in '0' '1' '2' '3' '4' '5' '6' '7' '8' '9'; do sed 1d data_final/owners_$filename.csv >> data_final/owners.csv; done;

In [2]:
owners = pd.read_csv('data_final/owners.csv')
owners = owners.drop_duplicates()
owners.to_csv('data_final/owners.csv', index = False)

In [ ]:
!aws s3 cp data_final/owners.csv s3://notebook-research/csv/owners.csv

## Repos

Download all repository csv files from s3

In [ ]:
combine_repos()


In [45]:
!head -1 data_final/repos_0.csv > data_final/repos.csv

In [46]:
!for filename in '0' '1' '2' '3' '4' '5' '6' '7' '8' '9'; do sed 1d data_final/repos_$filename.csv >> data_final/repos.csv; done;

In [13]:
repos = pd.read_csv('data_final/repos.csv')
print(len(repos) - len(repos.repo_id.unique()))
repos = repos.drop_duplicates()
print(len(repos) - len(repos.repo_id.unique()))
repos.to_csv('data_final/repos.csv', index = False)

41
0


In [ ]:
!aws s3 cp data_final/repos.csv s3://notebook-research/csv/repos.csv

## Notebooks

Download all notebooks csvs from s3

In [135]:
combine_notebooks(workers = [3])


0 1 2 3 4 5 No columns to parse from file
6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 
notebooks_3 loaded in 0:00:12.437477
notebooks_3 in csv in 0:00:08.408250
DONE! Now all in csv. 0:00:20.846297


In [143]:
!head -1 data_final/notebooks_0.csv > data_final/notebooks.csv
!for filename in '0' '1' '2' '3' '4' '5' '6' '7' '8' '9'; do sed 1d data_final/notebooks_$filename.csv >> data_final/notebooks.csv; echo $filename; done;

0
1
2
3
4
5
6
7
8
9


In [20]:
notebooks = pd.read_csv('data_final/notebooks.csv')
print(len(notebooks)-len(notebooks.file.unique()))
notebooks = notebooks.drop_duplicates()
print(len(notebooks)-len(notebooks.file.unique()))
notebooks.to_csv('data_final/notebooks.csv', index = False)

0
0


In [ ]:
!aws s3 cp data_final/notebooks.csv s3://notebook-research/csv/notebooks.csv

In [ ]:
!aws s3 cp data_final/notebooks_0.csv s3://notebook-research/csv/notebooks2_0.csv
!aws s3 cp data_final/notebooks_1.csv s3://notebook-research/csv/notebooks2_1.csv
!aws s3 cp data_final/notebooks_2.csv s3://notebook-research/csv/notebooks2_2.csv
!aws s3 cp data_final/notebooks_3.csv s3://notebook-research/csv/notebooks2_3.csv
!aws s3 cp data_final/notebooks_4.csv s3://notebook-research/csv/notebooks2_4.csv
!aws s3 cp data_final/notebooks_5.csv s3://notebook-research/csv/notebooks2_5.csv
!aws s3 cp data_final/notebooks_6.csv s3://notebook-research/csv/notebooks2_6.csv
!aws s3 cp data_final/notebooks_7.csv s3://notebook-research/csv/notebooks2_7.csv
!aws s3 cp data_final/notebooks_8.csv s3://notebook-research/csv/notebooks2_8.csv
!aws s3 cp data_final/notebooks_9.csv s3://notebook-research/csv/notebooks2_9.csv


## Cells

Combine all sub-csvs and save to 10 local csvs from s3

In [146]:
combine_cells(workers = [3])


0 No columns to parse from file
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 
cells_3 loaded in 0:03:56.867969
cells_3 in csv in 0:05:51.766928
DONE! Now all in csv. 0:09:48.635274


Concatenate these files

In [147]:
!head -1 data_final/cells_0.csv > data_final/cells.csv

In [148]:
!for filename in '0' '1' '2' '3' '4' '5' '6' '7' '8' '9'; do sed 1d data_final/cells_$filename.csv >> data_final/cells.csv; echo $filename; done;

0
1
2
3
4
5
6
7
8
9


In [ ]:
!aws s3 cp data_final/cells.csv s3://notebook-research/csv/cells.csv


In [ ]:
# !aws s3 cp data_final/cells_0.csv s3://notebook-research/csv/cells1_0.csv
# !aws s3 cp data_final/cells_1.csv s3://notebook-research/csv/cells1_1.csv
# !aws s3 cp data_final/cells_2.csv s3://notebook-research/csv/cells1_2.csv
!aws s3 cp data_final/cells_3.csv s3://notebook-research/csv/cells1_3.csv
# !aws s3 cp data_final/cells_4.csv s3://notebook-research/csv/cells1_4.csv
# !aws s3 cp data_final/cells_5.csv s3://notebook-research/csv/cells1_5.csv
# !aws s3 cp data_final/cells_6.csv s3://notebook-research/csv/cells1_6.csv
# !aws s3 cp data_final/cells_7.csv s3://notebook-research/csv/cells1_7.csv
# !aws s3 cp data_final/cells_8.csv s3://notebook-research/csv/cells1_8.csv
# !aws s3 cp data_final/cells_9.csv s3://notebook-research/csv/cells1_9.csv


# Remove files with missing data
Only keep notebooks that
- are downloaded
- have at least one line of code or one markdown word
- have repository metadata
- are *not* in ipynb_checkpoints

In [119]:
def clean_csvs(i):
    
    print('\nStarting',i)
    tracking = {
        'nbs': 0,
        'nbs_w_file': 0,
        'nbs_w_cells': 0,
        'nbs_w_repo': 0,
        'nbs_not_checkpoints': 0,
    }
    
    # Open CSVS
    t0 = datetime.datetime.now()

    cells = pd.read_csv('data_final/cells_{0}.csv'.format(i))
    t1 = datetime.datetime.now()
    print('cells', t1 - t0)

    repos = pd.read_csv('data_final/repos_{0}.csv'.format(i))
    t2 = datetime.datetime.now()
    print('repos', t2 - t1)

    notebooks = pd.read_csv('data_final/notebooks_{0}.csv'.format(i))
    t3 = datetime.datetime.now()
    print('notebooks', t3 - t2)

    owners = pd.read_csv('data_final/owners_{0}.csv'.format(i))
    t4 = datetime.datetime.now()
    print('owners', t4 - t3)
    
    # Missing Cell Content?
    nbs_w_file = set(notebooks.file)
    tracking['nbs'] = len(notebooks)
    tracking['nbs_w_file'] = len(nbs_w_file)

    df_nb_content_count = cells[["file", "lines_of_code", "num_words"]].groupby("file").sum().reset_index()
    nbs_w_cells = set(df_nb_content_count[np.logical_or(
            df_nb_content_count.lines_of_code != 0,
            df_nb_content_count.num_words != 0
        )].file.unique())
    tracking['nbs_w_cells'] = len(nbs_w_cells)
    
    # Missing Repo Data?
    repos_w_metadata = set(repos.repo_id.unique())
    nbs_w_repo = set(notebooks[notebooks.repo_id.isin(repos_w_metadata)]["file"])
    tracking['nbs_w_repo'] = len(nbs_w_repo)
    
    # In .ipynb checkpoints?
    nbs_not_checkpoints = set(notebooks[['ipynb_checkpoints' not in f for f in notebooks.file]]['file'])
    tracking['nbs_not_checkpoints'] = len(nbs_not_checkpoints)
    
    nbs_w_all = nbs_w_cells.intersection(nbs_w_file).intersection(nbs_w_repo).intersection(nbs_not_checkpoints)
    tracking['nbs_w_all'] = len(nbs_w_all)
    
    updated_notebooks = notebooks[notebooks.file.isin(nbs_w_all)]

    updated_repos = repos[repos.repo_id.isin(updated_notebooks.repo_id)]
    updated_cells = cells[cells.file.isin(nbs_w_all)]
    updated_owners = owners[owners.owner_id.isin(updated_repos.owner_id)]

    t0 = datetime.datetime.now()
    updated_notebooks.to_csv('data_final/notebooks_{0}_final.csv'.format(i), index = False)
    t1 = datetime.datetime.now()
    print('\nnotebooks saved', t1 - t0)
    updated_cells.to_csv('data_final/cells_{0}_final.csv'.format(i), index = False)
    t2 = datetime.datetime.now()
    print('cells saved', t2 - t1)
    updated_repos.to_csv('data_final/repos_{0}_final.csv'.format(i), index = False)
    t3 = datetime.datetime.now()
    print('repos saved', t3 - t2)
    updated_owners.to_csv('data_final/owners_{0}_final.csv'.format(i), index = False)
    t4 = datetime.datetime.now()
    print('owners saved', t4 - t3)
    
    print(i,'Done\n')
    
    return tracking

In [156]:
for i in [4,7]:
    all_tracking_dict[i] = clean_csvs(i)


Starting 4
cells 0:01:35.716037
repos 0:00:00.571895
notebooks 0:00:04.274490
owners 0:00:00.116801

notebooks saved 0:00:06.814475
cells saved 0:04:24.955093
repos saved 0:00:01.573472
owners saved 0:00:00.253961
4 Done


Starting

/home/ec2-user/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (9,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


 7
cells 0:02:19.097913
repos 0:00:00.577781
notebooks 0:00:04.483248
owners 0:00:00.118635

notebooks saved 0:00:07.144030
cells saved 0:03:50.268656
repos saved 0:00:01.580638
owners saved 0:00:00.249359
7 Done



In [160]:
print("{0}% were downloaded. {1}% had repo data downloaded. {2}% had at least one not-empty cell. {3}% weren't in ipynb_checkpoints.".format(
    round(100*sum([all_tracking_dict[i]['nbs_w_file'] for i in range(10)]) / sum([all_tracking_dict[i]['nbs'] for i in range(10)]),2),
    round(100*sum([all_tracking_dict[i]['nbs_w_repo'] for i in range(10)]) / sum([all_tracking_dict[i]['nbs'] for i in range(10)]),2),
    round(100*sum([all_tracking_dict[i]['nbs_w_cells'] for i in range(10)]) / sum([all_tracking_dict[i]['nbs'] for i in range(10)]),2),
    round(100*sum([all_tracking_dict[i]['nbs_not_checkpoints'] for i in range(10)]) / sum([all_tracking_dict[i]['nbs'] for i in range(10)]),2)
))
print("{0:,} ({1}%) of notebooks have all data.".format(
    sum([all_tracking_dict[i]['nbs_w_all'] for i in range(10)]),
    round(100*sum([all_tracking_dict[i]['nbs_w_all'] for i in range(10)]) / sum([all_tracking_dict[i]['nbs'] for i in range(10)]),2)
))

93.69% were downloaded. 93.44% had repo data downloaded. 92.01% had at least one not-empty cell. 76.77% weren't in ipynb_checkpoints.
4,290,766 (75.18%) of notebooks have all data.


93.69% were downloaded. 93.44% had repo data downloaded. 92.01% had at least one not-empty cell. 76.77% weren't in ipynb_checkpoints.
4,290,766 (75.18%) of notebooks have all data.

# Combine final versions and save to s3
'Final' meaning notebooks with all data

## Owners

In [161]:
!head -1 data_final/owners_0_final.csv > data_final/owners_final.csv
!for filename in $(ls data_final/owners_*_final.csv); do sed 1d $filename >> data_final/owners_final.csv; done;

In [26]:
owners_final = pd.read_csv('data_final/owners_final.csv')
print(len(owners_final) - len(owners_final.owner_id.unique()))
owners_final = owners_final.drop(columns = 'Unnamed: 0')
owners_final = owners_final.drop_duplicates()
print(len(owners_final) - len(owners_final.owner_id.unique()))
owners_final.to_csv('data_final/owners_final.csv', index = False)

327129
0


In [27]:
!aws s3 cp data_final/owners_final.csv s3://notebook-research/csv/owners_final.csv

upload: data_final/owners_final.csv to s3://notebook-research/csv/owners_final.csv


In [163]:
!aws s3 cp data_final/owners_0_final.csv s3://notebook-research/csv/owners2_0_final.csv
!aws s3 cp data_final/owners_1_final.csv s3://notebook-research/csv/owners2_1_final.csv
!aws s3 cp data_final/owners_2_final.csv s3://notebook-research/csv/owners2_2_final.csv
!aws s3 cp data_final/owners_3_final.csv s3://notebook-research/csv/owners2_3_final.csv
!aws s3 cp data_final/owners_4_final.csv s3://notebook-research/csv/owners2_4_final.csv
!aws s3 cp data_final/owners_5_final.csv s3://notebook-research/csv/owners2_5_final.csv
!aws s3 cp data_final/owners_6_final.csv s3://notebook-research/csv/owners2_6_final.csv
!aws s3 cp data_final/owners_7_final.csv s3://notebook-research/csv/owners2_7_final.csv
!aws s3 cp data_final/owners_8_final.csv s3://notebook-research/csv/owners2_8_final.csv
!aws s3 cp data_final/owners_9_final.csv s3://notebook-research/csv/owners2_9_final.csv


upload: data_final/owners_0_final.csv to s3://notebook-research/csv/owners2_0_final.csv
upload: data_final/owners_1_final.csv to s3://notebook-research/csv/owners2_1_final.csv
upload: data_final/owners_2_final.csv to s3://notebook-research/csv/owners2_2_final.csv
upload: data_final/owners_3_final.csv to s3://notebook-research/csv/owners2_3_final.csv
upload: data_final/owners_4_final.csv to s3://notebook-research/csv/owners2_4_final.csv
upload: data_final/owners_5_final.csv to s3://notebook-research/csv/owners2_5_final.csv
upload: data_final/owners_6_final.csv to s3://notebook-research/csv/owners2_6_final.csv
upload: data_final/owners_7_final.csv to s3://notebook-research/csv/owners2_7_final.csv
upload: data_final/owners_8_final.csv to s3://notebook-research/csv/owners2_8_final.csv
upload: data_final/owners_9_final.csv to s3://notebook-research/csv/owners2_9_final.csv


## Repos

In [164]:
!head -1 data_final/repos_0_final.csv > data_final/repos_final.csv
!for filename in $(ls data_final/repos_*_final.csv); do sed 1d $filename >> data_final/repos_final.csv; done;

In [35]:
repos_final = pd.read_csv('data_final/repos_final.csv')
print(len(repos_final) - len(repos_final.repo_id.unique()))
repos_final = repos_final.drop_duplicates()
print(len(repos_final) - len(repos_final.repo_id.unique()))
repos_final.to_csv('data_final/repos_final', index = False)

19
0


In [ ]:
!aws s3 cp data_final/repos_final.csv s3://notebook-research/csv/repos_final.csv

In [ ]:
!aws s3 cp data_final/repos_0_final.csv s3://notebook-research/csv/repos2_0_final.csv
!aws s3 cp data_final/repos_1_final.csv s3://notebook-research/csv/repos2_1_final.csv
!aws s3 cp data_final/repos_2_final.csv s3://notebook-research/csv/repos2_2_final.csv
!aws s3 cp data_final/repos_3_final.csv s3://notebook-research/csv/repos2_3_final.csv
!aws s3 cp data_final/repos_4_final.csv s3://notebook-research/csv/repos2_4_final.csv
!aws s3 cp data_final/repos_5_final.csv s3://notebook-research/csv/repos2_5_final.csv
!aws s3 cp data_final/repos_6_final.csv s3://notebook-research/csv/repos2_6_final.csv
!aws s3 cp data_final/repos_7_final.csv s3://notebook-research/csv/repos2_7_final.csv
!aws s3 cp data_final/repos_8_final.csv s3://notebook-research/csv/repos2_8_final.csv
!aws s3 cp data_final/repos_9_final.csv s3://notebook-research/csv/repos2_9_final.csv


## Notebooks

In [167]:
!head -1 data_final/notebooks_0_final.csv > data_final/notebooks_final.csv
!for filename in $(ls data_final/notebooks_*_final.csv); do sed 1d $filename >> data_final/notebooks_final.csv; done;

In [40]:
notebooks_final = pd.read_csv('data_final/notebooks_final.csv')
print(len(notebooks_final) - len(notebooks_final.file.unique()))
notebooks_final = notebooks_final.drop_duplicates()
print(len(notebooks_final) - len(notebooks_final.file.unique()))
notebooks_final.to_csv('data_final/notebooks_final.csv', index = False)

298892
0


In [ ]:
!aws s3 cp data_final/notebooks_final.csv s3://notebook-research/csv/notebooks_final.csv

In [ ]:
!aws s3 cp data_final/notebooks_0_final.csv s3://notebook-research/csv/notebooks2_0_final.csv
!aws s3 cp data_final/notebooks_1_final.csv s3://notebook-research/csv/notebooks2_1_final.csv
!aws s3 cp data_final/notebooks_2_final.csv s3://notebook-research/csv/notebooks2_2_final.csv
!aws s3 cp data_final/notebooks_3_final.csv s3://notebook-research/csv/notebooks2_3_final.csv
!aws s3 cp data_final/notebooks_4_final.csv s3://notebook-research/csv/notebooks2_4_final.csv
!aws s3 cp data_final/notebooks_5_final.csv s3://notebook-research/csv/notebooks2_5_final.csv
!aws s3 cp data_final/notebooks_6_final.csv s3://notebook-research/csv/notebooks2_6_final.csv
!aws s3 cp data_final/notebooks_7_final.csv s3://notebook-research/csv/notebooks2_7_final.csv
!aws s3 cp data_final/notebooks_8_final.csv s3://notebook-research/csv/notebooks2_8_final.csv
!aws s3 cp data_final/notebooks_9_final.csv s3://notebook-research/csv/notebooks2_9_final.csv


## Cells

In [4]:
!head -1 data_final/cells_0_final.csv > data_final/cells_final.csv
!for filename in $(ls data_final/cells_*_final.csv); do sed 1d $filename >> data_final/cells_final.csv; done;

In [ ]:
!aws s3 cp data_final/cells_final.csv s3://notebook-research/csv/cells_final.csv

In [ ]:
!aws s3 cp data_final/cells_0_final.csv s3://notebook-research/csv/cells1_0_final.csv
!aws s3 cp data_final/cells_1_final.csv s3://notebook-research/csv/cells1_1_final.csv
!aws s3 cp data_final/cells_2_final.csv s3://notebook-research/csv/cells1_2_final.csv
!aws s3 cp data_final/cells_3_final.csv s3://notebook-research/csv/cells1_3_final.csv
!aws s3 cp data_final/cells_4_final.csv s3://notebook-research/csv/cells1_4_final.csv
!aws s3 cp data_final/cells_5_final.csv s3://notebook-research/csv/cells1_5_final.csv
!aws s3 cp data_final/cells_6_final.csv s3://notebook-research/csv/cells1_6_final.csv
!aws s3 cp data_final/cells_7_final.csv s3://notebook-research/csv/cells1_7_final.csv
!aws s3 cp data_final/cells_8_final.csv s3://notebook-research/csv/cells1_8_final.csv
!aws s3 cp data_final/cells_9_final.csv s3://notebook-research/csv/cells1_9_final.csv


# Add to DataFrames

## Repos

### Add pushed to year and month

In [10]:
repos = pd.read_csv('data_final/repos_final.csv')

In [11]:
repos['pushed_at'] = pd.to_datetime(repos['pushed_at'])
repos['pushed_month'] = [p.month for p in repos['pushed_at']]
repos['pushed_year'] = [p.year for p in repos['pushed_at']]

In [13]:
repos = repos.drop(columns = ['Unnamed: 0'])

In [20]:
repos.to_csv('data_final/repos_final.csv', index = False)

In [ ]:
!aws s3 cp data_final/repos_final.csv s3://notebook-research/csv/repos_final.csv

## Notebooks

### Add Truncated Version

In [4]:
notebooks = pd.read_csv('data_final/notebooks_final.csv')

In [6]:
import datetime

In [7]:
start = datetime.datetime.now()

notebooks['lang_name'] = [str(n).lower() for n in notebooks['lang_name']]
python_versions_strings = notebooks[notebooks.lang_name == 'python'].lang_version
str_to_version = {}
for p in python_versions_strings.unique():
    if p == 'nan':
        str_to_version[p] = ''
    else:
        try:
            str_to_version[p] = float('.'.join(str(p).replace('+','').split('.')[0:2]))
        except:
            str_to_version[p] = ''

end = datetime.datetime.now()
print(end - start)

0:00:02.956888


In [9]:
# 11 minutes
start = datetime.datetime.now()

notebooks['python_version'] = [
    '' if p.lang_name != 'python' 
    else str_to_version[p.lang_version] 
    for i, p in notebooks.iterrows()
]

end = datetime.datetime.now()
print(end - start)


0:10:54.937147


### Add Pushed Date, Year, and Month

In [18]:
notebooks = notebooks.merge(repos[['repo_id','pushed_month','pushed_year']])

In [22]:
notebooks = notebooks.drop(columns = 'Unnamed: 0')

In [23]:
notebooks.head()

,file,google_collab,kernel_lang,kernel_name,lang_name,lang_version,nbformat,nbformat_minor,num_cells,days_since,filesize,html_url,name,owner_id,path,query_page,repo_id,python_version,pushed_month,pushed_year
0,artfin..NumerovProject..HO..diagpade_vs_centra...,False,python,Python 3,python,3.6.7,4.0,2.0,6.0,204.640719,107859..107926,https://github.com/artfin/NumerovProject/blob/...,plotter_k.ipynb,15310535,HO/diagpade_vs_central_diff/plotter_k.ipynb,1,126247677,3.6,3,2019
1,artfin..NumerovProject..HO..diagpade_vs_centra...,False,python,Python 3,python,3.6.7,4.0,2.0,7.0,204.679499,106810..106897,https://github.com/artfin/NumerovProject/blob/...,plotter.ipynb,15310535,HO/diagpade_vs_central_diff/plotter.ipynb,2,126247677,3.6,3,2019
2,artfin..NumerovProject..potential_profiles..po...,False,python,Python 3,python,3.6.7,4.0,2.0,5.0,204.820106,114679..114799,https://github.com/artfin/NumerovProject/blob/...,potential_profile.ipynb,15310535,potential_profiles/potential_profile.ipynb,3,126247677,3.6,3,2019
3,artfin..NumerovProject..dunker_pf_results..Rat...,False,python,Python 3,python,3.6.7,4.0,2.0,6.0,204.838960,115264..115375,https://github.com/artfin/NumerovProject/blob/...,Ratio of partition functions.ipynb,15310535,dunker_pf_results/Ratio of partition functions...,2,126247677,3.6,3,2019
4,artfin..NumerovProject..HO..numerov_vs_central...,False,python,Python 3,python,3.6.7,4.0,2.0,6.0,203.764665,98130..98247,https://github.com/artfin/NumerovProject/blob/...,plotter.ipynb,15310535,HO/numerov_vs_central_diff/plotter.ipynb,2,126247677,3.6,3,2019


In [24]:
notebooks.to_csv('data_final/notebooks_final.csv', index = False)

In [ ]:
!aws s3 cp data_final/notebooks_final.csv s3://notebook-research/csv/notebooks_final.csv